# Read in JADE Electron Energy Spectra for Ganymede from 1650 to 1659 UTC

""" SET UP
- import necessary libraries
"""


#Import libraries
import pandas as pd
import numpy as np
import scipy
from scipy import integrate
from scipy import interpolate
import glob
import re
import math
import os
#change to working directory
os.chdir('/Users/hwaite/Desktop/JUNO 2016 2022/JADEganymede/GanymedCodeWithInputFiles')

#  First version began in Fall 2021. Includes ion production from electron impact on H2, O2, and H2O
# as well as dissociative excitation of OI1305 and OI1356. Uses JADE data set from 165200 to 165899
# UTC. Cross sections from literature as noted below. Also uses Ben Teolis's atmospheric model with
# scaling inferred from Roth et al HST paper and enhanced O2 to match UVS PJ34 flyby.
#
# Updates listed in Early February: 1) Added JADE magnetopuase electron energy flux for aurora,
# 2) Added additional energy loss processes in O2 and H2 - Dissociation of O2 to fill energy loss 
# gap in the range of 40 to 100 eV, H2 & O2 vibrational excitation, O2 total scattering, 3) added
# in Maxwellians constrained by WAVES density and matched to JADE extrtapolated to 13 eV,
# 4) Added in photoionization rates from Huebner compilation for H2, O2, and H2O,5) Added in
# estimate of H2+ density using ion production and transport to use in determining the production
# of H3+, which also required adding in the JUNO occultation profile to get the ionospheric
# loss rates correct for H2+.

""" LOAD JADE DATA
--We have files from Frederic Allegrini of JADE data for 9 minute long averages during the Ganymede
--ionsopheric flyby 16:50:00 to 16:58:59
-- E: electron energy at the midpoint
-- PA1:  Pitch angle range 0 to 45 degrees
-- PA2:  Pitch angle range 45 to 135 degrees
-- PA3:  Pitch angle range 135 to 180 degrees
#WE HAVE ADDED THE FOLLOWING LOW ENERGY (13 eV) SLOPE EXTRAPOLATION TO THE PA1 COLUMN
#Calculate extrapolated JADE downward electron energy fluxes for low energy portion of the cross sections
#For O2 we have extra bins 13-15.5,15.5-18,18-23,23-28,28-32
#For H2 we have extra bins 17-20,-20-25,25-30,30-32
#For H2O we have extra bins 13.5-15,15-17.5,17.5-20,20-22.5,22.5-25,25-30,30-32
#For the different UTC's we have separate extrapolation slopes and lowest JADE energy measured 32.00-39.71 mid 35.86 eV
#Therefore, for the lowest energy point at 13 eV we get the following ==>
# 1652: -1.1295821 1.05e9 ==> 3.30e9
# 1653: -1.0375719 1.075e9 ==> 3.08e9
# 1654: -1.0320360 1.05e9 ==> 3.01e9
# 1655: -0.85439304 1.05e9 ==> 2.50e9
# 1656: -0.81824467 1.00e9 ==> 2.24e9
# 1657: -0.8566935 8.7e8 ==> 2.07e9
# 1658: -1.0945893 7.7e8 ==> 2.34e9


- We will add the following field to the JADE data:
-- Start time in UTC
"""
#Get filenames of each of the JADE data files
filenames = glob.glob('*.txt')
#print(filenames)
#Read each of the JADE data files
asJADE_header = ["E",  "PA1", "PA2", "PA3"]
list_of_dfs = [pd.read_csv(filename, names=asJADE_header) for filename in filenames]
#Add Start time (from each file name) to the corresponding dataframe
for dataframe, filename in zip(list_of_dfs, filenames):
    fileUTC = filename.split("_")[3]
    dataframe['UTC'] = fileUTC
#Concatenate all the JADE data file dataframes into a single dataframe
dfJADE_orig = pd.concat(list_of_dfs, ignore_index=True)
dfJADE_orig = dfJADE_orig.apply(pd.to_numeric)
#Grouping by Energy range producing the mean of the 9 minute time period 16:50:00 to 16:59:00 UTC
dfJADE_avByE = dfJADE_orig.groupby(["E"], as_index=False).mean()
dfJADE_sdByE = dfJADE_orig.groupby(["E"]).std()
#Need to delete the UTC column on both dataframes before merging and also need to relabel sd array to distinguish
dfJADE_avE = dfJADE_avByE.iloc[:,0:4]
dfJADE_sdE = dfJADE_sdByE.iloc[:,0:3]
#dfJADE_sdE.rename(columns={"PA1":"PA1sd","PA2":"PA2sd","PA3":"PA3sd"})
#dfJADE_Merge = pd.merge(dfJADE_avE, dfJADE_sdE, on="E", how="outer")
#print (dfJADE_orig)

#Read in Rob Ebert's magnetopause spectrum at 170100
aselecMP_header = ["EMP", "PMP", "UTC"]
dfelecMP = pd.read_csv('REganMPavg.csv',names=aselecMP_header,skiprows=1)
dfelecMP = dfelecMP.apply(pd.to_numeric)

#Rebecca plotting portion of code

import plotly.express as px
import plotly.graph_objects as go

dfJADE_orig.sort_values("UTC", inplace=True)
dfJADE_orig["UTC"] = dfJADE_orig["UTC"].astype(str)
fig1 = px.scatter(dfJADE_orig, x="E", y="PA1", title='JADE Electron Flux in Ganymede\'s Ionosphere', log_y=1, color="UTC")
fig1.update_yaxes(exponentformat="E")
fig1.show()
#fig2 = px.scatter(dfJADE_orig, x="E", y="PA2", title='JADE Electron Flux in Ganymede\'s Ionosphere', log_y=1, color="UTC")
#fig2.update_yaxes(exponentformat="E")
#fig2.show()
#fig3 = px.scatter(dfJADE_orig, x="E", y="PA3", title='JADE Electron Flux in Ganymede\'s Ionosphere', log_y=1, color="UTC")
#fig3.update_yaxes(exponentformat="E")
#fig3.show()

#fig_diff_1_3 = px.scatter(dfJADE_orig, x="E", y=dfJADE_orig["PA1"]-dfJADE_orig["PA3"], title='JADE Electron Flux in Ganymede\'s Ionosphere', log_y=1, color="UTC")
#fig_diff_1_3.update_yaxes(exponentformat="E", title='PA1-PA3')
#fig_diff_1_3.show()

#Grouping by Energy range producing the mean of the 9 min utc time period 16:50:00 to 16:59:00 UTC
#Note: UTC in resulting dfJADE_avByE or _sdByE is the avg or sd of the UTC times used

dfJADE_orig["UTC"] = dfJADE_orig["UTC"].astype('int32')

dfJADE_avByE = dfJADE_orig.groupby(["E"], as_index=False).mean()
dfJADE_sdByE = dfJADE_orig.groupby(["E"]).std()
#print(dfJADE_avByE)

#fig_diff_av1_av3 = px.scatter(dfJADE_avByE, x="E", y=dfJADE_avByE["PA1"]-dfJADE_avByE["PA3"], title='JADE Average Electron Flux in Ganymede\'s Ionosphere<br><sup>(Averaged over UTC 165000-165800)<sup>', log_y=1)
#fig_diff_av1_av3.update_yaxes(exponentformat="E", title='PA1-PA3')
#fig_diff_av1_av3.show()

dfJADE_avByE_0_5 = dfJADE_orig[dfJADE_orig.UTC < 165600].groupby(["E"], as_index=False).mean()
#print(dfJADE_avByE_0_5)

#fig_diff_av1_av3_files0_5 = px.scatter(dfJADE_avByE_0_5, x="E", y=dfJADE_avByE_0_5["PA1"]-dfJADE_avByE_0_5["PA3"], title='JADE Average Electron Flux in Ganymede\'s Ionosphere<br><sup>(Averaged over UTC 165000-165500)<sup>', log_y=1)
#fig_diff_av1_av3_files0_5.update_yaxes(exponentformat="E", title='PA1-PA3')
#fig_diff_av1_av3_files0_5.show()

dfJADE_avByE_6_8 = dfJADE_orig[dfJADE_orig.UTC >= 165600].groupby(["E"], as_index=False).mean()
#print(dfJADE_avByE_6_8)

#fig_diff_av1_av3_files6_8 = px.scatter(dfJADE_avByE_6_8, x="E", y=dfJADE_avByE_6_8["PA1"]-dfJADE_avByE_6_8["PA3"], title='JADE Average Electron Flux in Ganymede\'s Ionosphere<br><sup>(Averaged over UTC 165600-165800)<sup>', log_y=1)
#fig_diff_av1_av3_files6_8.update_yaxes(exponentformat="E", title='PA1-PA3')
#fig_diff_av1_av3_files6_8.show()


dfJADE_sub = dfJADE_orig[["E", "PA1", 'UTC']]
dfJADE_sub = dfJADE_sub.sort_values(by=['UTC','E'])
# Electron flux scaling factor
cortfac = 1.84e-3
print(dfJADE_sub)


#Define interpolation scheme to determine the electron flux at an arbitrary energy
#Use linear interpolation in a log flux/log energy parameter space
import pandas as pd
import numpy as np
import scipy
from scipy import integrate
from scipy import interpolate

#Define fit range and interpolate electron flux to energy e and change to per ev per 1.84 ster

def eflxforcs(e,xl,yl,Neval,Teval):
    if e<xl[0]:
        yint = eflxblw13(e,Neval,Teval)
        return yint
    else:
        x = np.log10(xl)
        y = np.log10(yl)
        enew = np.log10(e)
        f = interpolate.interp1d(x, y, kind = 'linear',fill_value ="extrapolate",assume_sorted=False)
        base = f(enew)
        yint = 10**base
        return yint

#Read in Electron Impact Ionization Cross sections

#Cross sections for H2, O2, and H2O in cm^2
asEIH2_header = ["E", "H2+", "H+"]
dfCSH2 = pd.read_csv('EI_H2_CS.csv',names=asEIH2_header,skiprows=1)
dfCSH2 = dfCSH2.apply(pd.to_numeric)
dfCSH2["H2+"] = dfCSH2["H2+"]*1.e-17
dfCSH2["H+"] = dfCSH2["H+"]*1.e-18
#print(dfCSH2)

#cross sections for O2 in cm^2
asEIO2_header = ["E", "O2+", "O+", "O++"]
dfCSO2 = pd.read_csv('EI_O2_CS.csv',names=asEIO2_header,skiprows=1)
dfCSO2 = dfCSO2.apply(pd.to_numeric)
dfCSO2["O2+"] = dfCSO2["O2+"]*1.e-16
dfCSO2["O+"] = dfCSO2["O+"]*1.e-17
dfCSO2["O++"] = dfCSO2["O++"]*1.e-18
#print(dfCSO2)

#Cross sections for H2O in cm^2
#The H2O+ includes OH+ and O+ as well as H2O+
asEIH2O_header = ["E", "H2O+", "O++", "H2+", "H+"]
dfCSH2O = pd.read_csv('EI_H2O_CS.csv',names=asEIH2O_header,skiprows=1)
dfCSH2O = dfCSH2O.apply(pd.to_numeric)
dfCSH2O["H2O+"] = dfCSH2O["H2O+"]*1.e-16
dfCSH2O["O++"] = dfCSH2O["O++"]*1.e-19
dfCSH2O["H2+"] = dfCSH2O["H2+"]*1.e-19
dfCSH2O["H+"] = dfCSH2O["H+"]*1.e-17
#print(dfCSH2O)

#Read in Dissociative Excitation Cross sections

#Electron impact Dissociative Excitation Cross sections for O2 and H2O in cm^2

#cross sections for O2 in cm^2
#O2 DisEx comes from Kanik et al.JGR, 108(E11),5126, doi:10.1029/2000JE001423,2003

#e- on O2 --> 1304
#Uses Table2 to 600 eV then Aartes and De Heers [1971] column scaled to go to higher energies
# and finally extends slope using Aartes from 1000 to 5000 eV to reach end of file energy
asDEO21304_header = ["EO21304", "CS1304Mb"]
dfCSdeO21304 = pd.read_csv('DisExeO21304.csv',names=asDEO21304_header,skiprows=1)
dfCSdeO21304 = dfCSdeO21304.apply(pd.to_numeric)
dfCSdeO21304["CS1304Mb"] = dfCSdeO21304["CS1304Mb"]*1.e-18
#e- on O2 --> 1356
#Uses Table3 to 600 eV then follows shape function of 1304 above to end of eenergy range of file
asDEO21356_header = ["EO21356", "CS1356Mb"]
dfCSdeO21356 = pd.read_csv('DisExeO21356.csv',names=asDEO21356_header,skiprows=1)
dfCSdeO21356 = dfCSdeO21356.apply(pd.to_numeric)
dfCSdeO21356["CS1356Mb"] = dfCSdeO21356["CS1356Mb"]*1.e-18

#print(dfCSdeO21304,dfCSdeO21356)

#Cross sections for H2O in cm^2
#H2O DisEx comes from Makarov et al., JGR, 109, A09303, doi:10.1029/2002JA009353, 2004
#e- on H2O --> 1304
#Uses Table 3 and extends using O2 shape function as indicated above to higher energy
asDEH2O1304_header = ["EH2O1304", "CS1304Mb$10"]
dfCSdeH2O1304 = pd.read_csv('DisExeH2O1304.csv',names=asDEH2O1304_header,skiprows=1)
dfCSdeH2O1304 = dfCSdeH2O1304.apply(pd.to_numeric)
dfCSdeH2O1304["CS1304Mb$10"] = dfCSdeH2O1304["CS1304Mb$10"]*1.e-19
#e- on O2 --> 1356
#Uses Table 1 to get ratio of total 1304 to 1356 and folows the 1304 shape function
asDEH2O1356_header = ["EH2O1356", "CS1356Mb$10"]
dfCSdeH2O1356 = pd.read_csv('DisExeH2O1356.csv',names=asDEH2O1356_header,skiprows=1)
dfCSdeH2O1356 = dfCSdeH2O1356.apply(pd.to_numeric)
dfCSdeH2O1356["CS1356Mb$10"] = dfCSdeH2O1356["CS1356Mb$10"]*1.e-20
#print(dfCSdeH2O1304,dfCSdeH2O1356)

#Read in cross sections for other e on O2 processes to use to check production or
#energy balance and loss
asQTeo2_header = ["EO2QT", "CSQTeO2"]
dfCSQTo2 = pd.read_csv('QTscatO2.csv',names=asQTeo2_header,skiprows=1)
dfCSQTo2 = dfCSQTo2.apply(pd.to_numeric)
asvibeo2_header = ["EO2vib", "CSvibeO2"]
dfCSvibo2 = pd.read_csv('O2 Vib.csv',names=asvibeo2_header,skiprows=1)
dfCSvibo2 = dfCSvibo2.apply(pd.to_numeric)
asB3sigeo2_header = ["EO2B3sig", "CSB3sigeO2"]
dfCSB3sigo2 = pd.read_csv('BtripletSigmaDissociation.csv',names=asB3sigeo2_header,skiprows=1)
dfCSB3sigo2 = dfCSB3sigo2.apply(pd.to_numeric)
asvibh2_header = ["EH2vib", "CSvibeH2"]
dfCSvibh2 = pd.read_csv('eCSH2vibSchulzWestinghouse.csv',names=asvibh2_header,skiprows=1)
dfCSvibh2 = dfCSvibh2.apply(pd.to_numeric)

#Table of Maxwellian values for thermal electrons at a given UTC
asMaxwell_header = ["UTC","Ne","Te"]
dfMaxIn = pd.read_csv('Maxwellin.csv', names=asMaxwell_header,skiprows=1)
dfMaxIn = dfMaxIn.apply(pd.to_numeric)
def eflxblw13(e,Neval,Teval):
    rkbev = 8.6171e-5
    rkberg = 1.3805e-16
    elecmass = 9.1091e-28
    elecvel = math.sqrt(2.*rkberg*Teval/elecmass)
    ni = Neval*np.exp(-e/(rkbev*Teval))
    eflux = ni/4./3.1415*elecvel*1.84
    return eflux



#Read in atmospheric densities as a afunction of altitude in meters
asAtm_header = ["alt", "H2On", "O2n", "H2n", "H2Om", "O2m", "H2m"]
dfAtm = pd.read_csv('Ganymede_09092021_Audrey_input2.csv',names=asAtm_header,skiprows=3)
dfAtm = dfAtm.apply(pd.to_numeric)
print(dfAtm)

#Calculate the mean free path in the atmosphere the surface to 5000 km.according to the altitude profile given in the Teolis
#spreadsheet.
#The formula for mean free path is given by λ= 1/(sqrt(2)*pi*d^2*n  
#n is the astmospheric density, pi*d^2 = 4e-19 m^2 O2 effective cross sectional area for particle
#of radius d/2
#The scale factors for night and day are shown in green based on Roth et al HST observations and auroal emission constraints

import numpy as np
altgrd = np.array(dfAtm['alt'])
nh2oden = np.array(dfAtm['H2On'])
no2den = np.array(dfAtm['O2n'])*20.
nh2den = np.array(dfAtm['H2n'])
ntden = nh2oden+no2den+nh2den
nden = np.array([nh2den,no2den,nh2oden])
mh2oden = np.array(dfAtm['H2Om'])
mo2den = np.array(dfAtm['O2m'])
mh2den = np.array(dfAtm['H2m'])
mtden = mh2oden+no2den+nh2den
nmfpm = (1./(2**0.5*4.e-19*ntden))
mmfpm = (1./(2**0.5*4.e-19*mtden))
dfMFP = pd.DataFrame(data={'ALTITUDE': altgrd, 'noon mean free path m': nmfpm, 'midnight mean free path m': mmfpm})
#Plot mean free path in km versus altitude in kilometers on the dayside                     
figmfp = px.scatter(x=nmfpm/1000.,y=altgrd/1000.,log_x=1,title="Mean Free Path (km)",labels={"y":  "Altitude(km)", "x": "Mean Free Path(km)"})
figmfp.update_xaxes(exponentformat='E', title='Mean Free Path (km)')
figmfp.show()
#Plot for displaying the atmospheric density vs altitude
#noon
figatmn = px.scatter(x=no2den/1.E6, y=altgrd/1000., log_x=1, title='Atmospheric densities (cm^-3)',labels={"y":  "Altitude(km)", "x": "Density(cm^-3)"})
figatmn.add_scatter(x=no2den/1.E6, y=altgrd/1000., mode='markers',name='Day O2')
figatmn.add_scatter(x=nh2oden/1.E6, y=altgrd/1000., mode='markers', name='Day H2O')
figatmn.add_scatter(x=nh2den/1.E6, y=altgrd/1000.,  mode='markers',name='Day H2')
figatmn.update_xaxes(exponentformat="E")
figatmn.show()
# Determine atmospheric column densities on dayside
colnh2oden = integrate.simpson(nh2oden,altgrd,even='avg')
colno2den = integrate.simpson(no2den,altgrd,even='avg')
colnh2den = integrate.simpson(nh2den,altgrd,even='avg')
print("NOON Column (m^-2)", "H2O =", "{:.2e}".format(colnh2oden), "O2", "{:.2e}".format(colno2den),"H2", "{:.2e}".format(colnh2den))
#midnight
figatmm = px.scatter(x=mo2den/1.E6, y=altgrd/1000., log_x=1, title='Atmospheric densities (cm^-3)',labels={"y":  "Altitude(km)", "x": "Density(cm^-3)"})
figatmm.add_scatter(x=mo2den/1.E6, y=altgrd/1000., mode='markers',name='Night O2')
figatmm.add_scatter(x=mh2oden/1.E6, y=altgrd/1000., mode='markers', name='Night H2O')
figatmm.add_scatter(x=mh2den/1.E6, y=altgrd/1000.,  mode='markers',name='Night H2')
figatmm.update_xaxes(exponentformat="E")
figatmm.show()
# Deternmmine column densities on the nightside
colmh2oden = integrate.simpson(mh2oden,altgrd,even='avg')
colmo2den = integrate.simpson(mo2den,altgrd,even='avg')
colmh2den = integrate.simpson(mh2den,altgrd,even='avg')

print("MIDNIGHT Column (m^-2)", "H2O =", "{:.2e}".format(colmh2oden), "O2", "{:.2e}".format(colmo2den),"H2", "{:.2e}".format(colmh2den))

# Initialize kinetic rate array
iterkr = 38-1
KR = np.zeros(iterkr)
print(KR)
        
#Function to determine the sample ionospheric density in m-3 from Dustin ingress n0=2250cm-3 T=880 for O2+
# up to 150 km and above that 880K for an equivalant mass of 8.6 amu
def electrondensity(altgrd):
    eden = np.zeros_like(altgrd)
    for j in range(len(altgrd-1)):
        z = altgrd[j]
        if z<1.50e5:
            eden[j] = 2.2e7*np.exp(-z/3.28e5)
        else:
            eden[j] = 1.4e7*np.exp(-z/1.23e6)
        if eden[j]<1.:
            eden[j] = 1.
    return eden

#Function to adjust for fast transit of H2+ through altitude grid cell taking VSet from JADE data
def ttc(altgrd,h2plf):
    vset = 150.
    transadjust = np.zeros_like(altgrd)
    grdstep = np.zeros_like(altgrd)
    jtop = len(altgrd-1)
    for j in range(0,jtop):
        grdstep[j] = altgrd[j+1] - altgrd[j]
        transadjust[j] = (grdstep[j]/vset)/(1./h2plf[j])
        if transadjust[j] > 1.0:
            transadjust[j] = 1.
    transadjust[jtop] = transadjust[jtop-1]
    figtrans = px.scatter(x=transadjust, y=altgrd/1000., log_x=1, title='Transadjust',labels={"y":  "Altitude (km)", "x": "Transadjust"})
    figtrans.add_scatter(x=1./h2plf, y=altgrd/1000., mode='markers',name='H2+lifetime(s)')
    figtrans.update_xaxes(exponentformat="E")
    figtrans.show()
    return transadjust
    

#Function to determine H2+ density versus altitude
def H2iondensity(altgrd,h2den,adph2oh2p,adph2h2p,photh2h2p):
    h2pden = np.zeros_like(altgrd)
    h2plf = np.zeros_like(altgrd)
    h2plf = np.array((1.6e-14*(300./880.)**0.4*electrondensity(altgrd)) + h2den*2.e-15)
    transadj = np.array(ttc(altgrd,h2plf))
    h2pden = np.array((adph2oh2p + adph2h2p + photh2h2p)/(h2plf*transadj))
    return h2pden

#Cross check the expected photolysis production using Huebner reference
photh2h2p = np.array(nh2den*5.41e-8/27.04)
photh2hp = np.array(nh2den*9.52e-9/27.04)
photo2o2p = np.array(no2den*4.67e-7/27.04)
photo2op = np.array(no2den*1.1e-7/27.04)
photh2ohp = np.array(nh2oden*1.31e-8/27.04)
photh2oh2op = np.array(nh2oden*3.31e-7/27.04)
photh2oop = np.array(nh2oden*5.85e-9/27.04)
photh2oohp = np.array(nh2oden*5.54e-8/27.04)
#Plot photolysis production rates
figphot = px.scatter(x=photo2o2p/1.E6, y=altgrd/1000., log_x=1, title='Photolysis production rates (cm^-3 s^-1)',labels={"y":  "Altitude (km)", "x": "Photolysis Production (cm^-3 S^-1)"})
figphot.add_scatter(x=photo2o2p/1.E6, y=altgrd/1000., mode='markers',name='O2+')
figphot.add_scatter(x=photh2h2p/1.E6, y=altgrd/1000., mode='markers', name='H2+')
figphot.add_scatter(x=(photh2hp+photh2ohp)/1.E6, y=altgrd/1000.,  mode='markers',name='H+')
figphot.add_scatter(x=(photo2op+photh2oh2op+photh2oop+photh2oohp)/1.E6, y=altgrd/1000.,  mode='markers',name='WG+')
figphot.update_xaxes(exponentformat="E")
figphot.show()

#column integrate
cphoth2h2p = integrate.simpson(photh2h2p,altgrd,even='avg')
cphoth2hp = integrate.simpson(photh2hp,altgrd,even='avg')
cphoto2o2p = integrate.simpson(photo2o2p,altgrd,even='avg')
cphoto2op = integrate.simpson(photo2op,altgrd,even='avg')
cphoth2ohp = integrate.simpson(photh2ohp,altgrd,even='avg')
cphoth2oh2op = integrate.simpson(photh2oh2op,altgrd,even='avg')
cphoth2oop = integrate.simpson(photh2oop,altgrd,even='avg')
cphoth2oohp = integrate.simpson(photh2oohp,altgrd,even='avg')
#mass 2
print('Photolysis producing H2+',"{:.2e}".format(cphoth2h2p))
#mass 1
cphothp = cphoth2hp + cphoth2ohp
print('Photolysis producing H+',"{:.2e}".format(cphothp))
#mass ~16
cphotm16 = cphoto2op + cphoth2oh2op + cphoth2oop + cphoth2oohp
print('Photolysis producing m16+',"{:.2e}".format(cphotm16))
# mass 32
print('Photolysis producing O2+',"{:.2e}".format(cphoto2o2p))
                    
#Determine integrated production rates at each UTC for H2, O2, and H2O as a function of altitude
#Calculate the altitude dependent production rates assuming the downgoing electrons are the correct electron
#fluxes all the way to the ground since the atmospheric densities are extremely low and the mean free path is kilometers.


                  
# begin loop over UTC
ProdO2Ndf = pd.DataFrame()
ProdH2Ndf = pd.DataFrame()
ProdH2ONdf = pd.DataFrame()
ProdO2Mdf = pd.DataFrame()
ProdH2Mdf = pd.DataFrame()
ProdH2OMdf = pd.DataFrame()
CProdO2Ndf = pd.DataFrame()
CProdH2Ndf = pd.DataFrame()
CProdH2ONdf = pd.DataFrame()
CProdO2Mdf = pd.DataFrame()
CProdH2Mdf = pd.DataFrame()
CProdH2OMdf = pd.DataFrame()
UTCar = [165200,165300,165400,165500,165600,165700,165800,170100]
UTCidx = len(UTCar)
idx=0
print (UTCidx,UTCar)
for idx in range(UTCidx):
    nutc = UTCar[idx]
    print(nutc)
    if nutc==170100:
        dfenergy = np.array(dfelecMP[dfelecMP.UTC == nutc].EMP,dtype=float)
        dfelecflux =np.array(dfelecMP[dfelecMP.UTC == nutc].PMP*cortfac,dtype=float)
        print(dfenergy)
        print(dfelecflux)
    else:
        dfenergy = np.array(dfJADE_sub[dfJADE_sub.UTC == nutc].E,dtype=float)
        dfelecflux = np.array(dfJADE_sub[dfJADE_sub.UTC == nutc].PA1*cortfac,dtype=float)
        sumo = 0.      
        sum11 = 0.
        sum12 = 0.
        sum13 = 0.
        sum14 = 0.
        sum15 = 0.
        sum16 = 0.
        sum17 = 0.   
#Photoelectron correction for low optical depth of electrons from spreadsheet on Opal, Peterson, et al. paper
        sumo = (dfelecflux[3]+dfelecflux[4]+dfelecflux[5])*0.091
        sum11 = (dfelecflux[6]+dfelecflux[7]+dfelecflux[8]+dfelecflux[9])*0.25
        sum12 = (dfelecflux[10]+dfelecflux[11]+dfelecflux[12]+dfelecflux[13])*0.25
        sum13 = (dfelecflux[14]+dfelecflux[15]+dfelecflux[16]+dfelecflux[17])*0.35
        sum14 = (dfelecflux[18]+dfelecflux[19])*0.35
        sum15 = (dfelecflux[20]+dfelecflux[21]+dfelecflux[22]+dfelecflux[23]+dfelecflux[24])*0.45                          
        sum16 = (dfelecflux[25]+dfelecflux[26]+dfelecflux[27]+dfelecflux[28]+dfelecflux[29])*0.45
        sum17 = (dfelecflux[30]+dfelecflux[31]+dfelecflux[32])*0.45
        esumo = sumo*13.
        sum1 = (sum11+sum12+sum13+sum14+sum15+sum16+sum17)
        esum1 = sum1*35.9
        print("PE ENERGY SUMS","{:.2e}".format(esumo*1e4*1.6e-16),"{:.2e}".format(esum1*1e4*1.6e-16))
        dfelecflux[0] =dfelecflux[0] + sumo
        dfelecflux[1] = dfelecflux[1] + sum1
    Teval = (dfMaxIn[dfMaxIn.UTC == nutc].Te)
    Neval = (dfMaxIn[dfMaxIn.UTC == nutc].Ne)
    xpass = np.array(dfenergy)
    ypass = np.array(dfelecflux)
#    print(xpass,ypass)
#    type(xpass)
#   type(ypass)
   
                  
#Calculate energy dependent production rates
#These do not depend on altitude or day vs night since we assume optically thin electron influx conditions
                  
#ionization of O2
    eno2 = np.array(dfCSO2["E"],dtype=float)
    cso2o2p = np.array(dfCSO2["O2+"],dtype=float)
    cso2op = np.array(dfCSO2["O+"],dtype=float)
    cso2opp = np.array(dfCSO2["O++"],dtype=float)
#    print(eno2,cso2o2p,cso2op)
#log interpolate the electron energy flux at energies eno2 and load into array elecflux
    nepts = len(eno2)
    csefo2o2p = np.zeros_like(cso2o2p)
    csefo2op = np.zeros_like(cso2op)
    csefo2opp = np.zeros_like(cso2opp)
    for j in range(nepts):
        e = eno2[j]
        ef = eflxforcs(e,xpass,ypass,Neval,Teval)
        csefo2o2p[j] = ef*cso2o2p[j]
        csefo2op[j] = ef*cso2op[j]
        csefo2opp[j] = ef*cso2opp[j]
    pfo2o2p = integrate.simpson(csefo2o2p,eno2,even='avg')
    KR[36] = pfo2o2p
#    print(pfo2o2p)
    pfo2op = integrate.simpson(csefo2op,eno2,even='avg')
    KR[30] = pfo2op
#    print(pfo2op)
    pfo2opp =integrate.simpson(csefo2opp,eno2,even='avg')
#    print(pfo2opp)

#ionization of H2
    enh2 = np.array(dfCSH2["E"],dtype=float)
    csh2h2p = np.array(dfCSH2["H2+"],dtype=float)
    csh2hp = np.array(dfCSH2["H+"],dtype=float)
    nepts = len(enh2)
    csefh2h2p = np.zeros_like(csh2h2p)
    csefh2hp = np.zeros_like(csh2hp)
    csefo2opp = np.zeros_like(cso2opp)
    for j in range(nepts):
        e = enh2[j]
        ef = eflxforcs(e,xpass,ypass,Neval,Teval)
        csefh2h2p[j] = ef*csh2h2p[j]
        csefh2hp[j] = ef*csh2hp[j]
    pfh2h2p = integrate.simpson(csefh2h2p,enh2,even='avg')
    KR[5] = pfh2h2p
#    print(pfh2h2p)
    pfh2hp = integrate.simpson(csefh2hp,enh2,even='avg')
    KR[1] = pfh2hp
#    print(pfh2hp)

# ionization of H2O
    enh2o = np.array(dfCSH2O["E"],dtype=float)
    csh2oh2op = np.array(dfCSH2O["H2O+"],dtype=float)
    csh2oopp = np.array(dfCSH2O["O++"],dtype=float)
    csh2oh2p = np.array(dfCSH2O["H2+"],dtype=float)
    csh2ohp = np.array(dfCSH2O["H+"],dtype=float)
    nepts = len(enh2o)
    csefh2oh2op = np.zeros_like(csh2oh2op)
    csefh2oopp = np.zeros_like(csh2oopp)
    csefh2oh2p = np.zeros_like(csh2oh2p)
    csefh2ohp = np.zeros_like(csh2ohp)
    for j in range(nepts):
        e = enh2o[j]
        ef = eflxforcs(e,xpass,ypass,Neval,Teval)
        csefh2oh2op[j] = ef*csh2oh2op[j]
        csefh2oopp[j] = ef*csh2oopp[j]
        csefh2oh2p[j] = ef*csh2oh2p[j]
        csefh2ohp[j] = ef*csh2ohp[j]
    pfh2oh2op = integrate.simpson(csefh2oh2op,enh2o,even='avg')
    KR[15] = pfh2oh2op
#    print(pfh2oh2op)
    pfh2oopp = integrate.simpson(csefh2oopp,enh2o,even='avg')
#    print(pfh2oo2p)
    pfh2oh2p = integrate.simpson(csefh2oh2p,enh2o,even='avg')
#    print(pfh2oh2p)
    KR[23]
    pfh2ohp = integrate.simpson(csefh2ohp,enh2o,even='avg')
    KR[21] = pfh2ohp
#    print(pfh2ohp)
                     
#dissociative excitation of O2
    eno21304 = np.array(dfCSdeO21304["EO21304"],dtype=float)
    cso21304 = np.array(dfCSdeO21304["CS1304Mb"],dtype=float)
    nepts = len(eno21304)
    csefo21304 = np.zeros_like(cso21304)
    for j in range(nepts):
        e = eno21304[j]
        ef = eflxforcs(e,xpass,ypass,Neval,Teval)
        csefo21304[j] = ef*cso21304[j]
    pfo21304 = integrate.simpson(csefo21304,eno21304,even='avg')
#    print(pfo21304)
    eno21356 = np.array(dfCSdeO21356["EO21356"],dtype=float)
    cso21356 = np.array(dfCSdeO21356["CS1356Mb"],dtype=float)
    nepts = len(eno21356)
    csefo21356 = np.zeros_like(cso21356)
    for j in range(nepts):
        e = eno21356[j]
        ef = eflxforcs(e,xpass,ypass,Neval,Teval)
        csefo21356[j] = ef*cso21356[j]
    pfo21356 = integrate.simpson(csefo21356,eno21356,even='avg')
#    print(pfo21356)

#dissociative excitation of H2O
    enh2o1304 = np.array(dfCSdeH2O1304["EH2O1304"],dtype=float)
    csh2o1304 = np.array(dfCSdeH2O1304["CS1304Mb$10"],dtype=float)
    nepts = len(enh2o1304)
    csefh2o1304 = np.zeros_like(csh2o1304)
    for j in range(nepts):
        e = enh2o1304[j]
        ef = eflxforcs(e,xpass,ypass,Neval,Teval)
        csefh2o1304[j] = ef*csh2o1304[j]
    pfh2o1304 = integrate.simpson(csefh2o1304,enh2o1304,even='avg')
#    print(pfh2o1304)
    enh2o1356 = np.array(dfCSdeH2O1356["EH2O1356"],dtype=float)
    csh2o1356 = np.array(dfCSdeH2O1356["CS1356Mb$10"],dtype=float)
    nepts = len(enh2o1356)
    csefh2o1356 = np.zeros_like(csh2o1356)
    for j in range(nepts):
        e = enh2o1356[j]
        ef = eflxforcs(e,xpass,ypass,Neval,Teval)
        csefh2o1356[j] = ef*csh2o1356[j]
    pfh2o1356 = integrate.simpson(csefh2o1356,enh2o1356,even='avg')
#    print(pfh2o1356) 

#calculate the column dissociation of O2 from the BtripletSigma cross section
    eno2dis = np.array(dfCSB3sigo2["EO2B3sig"],dtype=float)
    cso2dis = np.array(dfCSB3sigo2["CSB3sigeO2"],dtype=float)
    nepts = len(eno2dis)
    csefo2dis = np.zeros_like(cso2dis)
    for j in range(nepts):
        e = eno2dis[j]
        ef = eflxforcs(e,xpass,ypass,Neval,Teval)
        csefo2dis[j] = ef*cso2dis[j]
    pfo2dis = integrate.simpson(csefo2dis,eno2dis,even='avg')

#calculate the vibrational excitation of O2 and H2
    eno2vib = np.array(dfCSvibo2["EO2vib"],dtype=float)
    cso2vib = np.array(dfCSvibo2["CSvibeO2"],dtype=float)
    nepts = len(eno2vib)
    csefo2vib = np.zeros_like(cso2vib)
    for j in range(nepts):
        e = eno2vib[j]
        ef = eflxforcs(e,xpass,ypass,Neval,Teval)
        csefo2vib[j] = ef*cso2vib[j]
    pfo2vib = integrate.simpson(csefo2vib,eno2vib,even='avg')
    enh2vib = np.array(dfCSvibh2["EH2vib"],dtype=float)
    csh2vib = np.array(dfCSvibh2["CSvibeH2"],dtype=float)
    nepts = len(enh2vib)
    csefh2vib = np.zeros_like(csh2vib)
    for j in range(nepts):
        e = enh2vib[j]
        ef = eflxforcs(e,xpass,ypass,Neval,Teval)
        csefh2vib[j] = ef*csh2vib[j]
    pfh2vib = integrate.simpson(csefh2vib,enh2vib,even='avg')
    
#calculate the total scattering from O2
    eno2scat = np.array(dfCSQTo2["EO2QT"],dtype=float)
    cso2scat = np.array(dfCSQTo2["CSQTeO2"],dtype=float)
    nepts = len(eno2scat)
    csefo2scat = np.zeros_like(cso2scat)
    for j in range(nepts):
        e = eno2scat[j]
        ef = eflxforcs(e,xpass,ypass,Neval,Teval)
        csefo2scat[j] = ef*cso2scat[j]
    pfo2scat = integrate.simpson(csefo2scat,eno2scat,even='avg')
                  
#Solve sperately for dayside(n) and nightside(m)
                  
#Iterate over altitude
#for O2 processes
    nadpo2o2p = np.array(no2den*pfo2o2p)
    noprods8 = np.array(nadpo2o2p)
    madpo2o2p = np.array(mo2den*pfo2o2p)
    moprods8 = np.array(madpo2o2p)
    nadpo2op = np.array(no2den*pfo2op)
    noprods4 = np.array(nadpo2op)
    madpo2op = np.array(mo2den*pfo2op)
    moprods4 = np.array(madpo2op)
    nadpo2opp = np.array(no2den*pfo2opp)
    madpo2opp = np.array(mo2den*pfo2opp)
    nadpo21304 = np.array(no2den*pfo21304)
    madpo21304 = np.array(mo2den*pfo21304)
    nadpo21356 = np.array(no2den*pfo21356)
    madpo21356 = np.array(mo2den*pfo21356)
    nadpo2dis = np.array(no2den*pfo2dis)
    madpo2dis = np.array(mo2den*pfo2dis)
    nadpo2vib = np.array(no2den*pfo2vib)
    madpo2vib = np.array(mo2den*pfo2vib)
    nadpo2scat = np.array(no2den*pfo2scat)
    madpo2scat = np.array(mo2den*pfo2scat)
    ProdO2Ndf_iteration = pd.DataFrame(data={'UTC':nutc,'ALT':altgrd,'NADPO2O2P':nadpo2o2p,'NADPO2OP':nadpo2op,'NADPO2OPP':nadpo2opp,'NADPO21304':nadpo21304,'NADPO21356':nadpo21356,'NADPO2dis':nadpo2dis,'NADPO2vib':nadpo2vib,'NADPO2scat':nadpo2scat})
    ProdO2Mdf_iteration = pd.DataFrame(data={'UTC':nutc,'ALT':altgrd,'MADPO2O2P':madpo2o2p,'MADPO2OP':madpo2op,'MADPO2OPP':madpo2opp,'MAPDO21304':madpo21304,'MADPO21356':madpo21356,'MADPO2dis':madpo2dis,'MADPO2vib':madpo2vib,'MADPO2scat':madpo2scat})
    ProdO2Ndf = ProdO2Ndf.append(ProdO2Ndf_iteration,ignore_index = True)
    ProdO2Mdf = ProdO2Mdf.append(ProdO2Mdf_iteration,ignore_index = True)

    
#for H2O processes
    nadph2oh2op = np.array(nh2oden*pfh2oh2op)
    noprods6 = np.array(nadph2oh2op)
    madph2oh2op = np.array(mh2oden*pfh2oh2op)
    moprods4 = np.array(moprods4+madph2oh2op)
    nadph2oopp = np.array(nh2oden*pfh2oopp)
    madph2oopp = np.array(mh2oden*pfh2oopp)
    nadph2oh2p = np.array(nh2oden*pfh2oh2p)
    noprods2 = np.array(nadph2oh2p)
    madph2oh2p = np.array(mh2oden*pfh2oh2p)
    moprods2 = np.array(madph2oh2p)
    nadph2ohp = np.array(nh2oden*pfh2ohp)
    noprods1 = np.array(nadph2ohp)
    madph2ohp = np.array(mh2oden*pfh2ohp)
    moprods1 = np.array(madph2ohp)
    nadph2o1304 = np.array(nh2oden*pfh2o1304)
    madph2o1304 = np.array(mh2oden*pfh2o1304)
    nadph2o1356 = np.array(nh2oden*pfh2o1356)
    madph2o1356 = np.array(mh2oden*pfh2o1356)
    ProdH2ONdf_iteration = pd.DataFrame(data={'UTC':nutc,'ALT':altgrd,'NADPH2OH2OP':nadph2oh2op,'NADPH2OOPP':nadph2oopp,'NADPH2OH2P':nadph2oh2p,'NADPH2OHP':nadph2ohp,'NAPDH2O1304':nadph2o1304,'NAPDH2O1356':nadph2o1356})
    ProdH2OMdf_iteration = pd.DataFrame(data={'UTC':nutc,'ALT':altgrd,'MADPH2OH2OP':madph2oh2op,'MADPH2OOPP':madph2oopp,'MADPH2OH2P':madph2oh2p,'MADPH2OHP':madph2ohp,'MAPDH2O1304':madph2o1304,'MAPDH2O1356':madph2o1356})
    ProdH2ONdf = ProdH2ONdf.append(ProdH2ONdf_iteration,ignore_index = True)
    ProdH2OMdf = ProdH2OMdf.append(ProdH2OMdf_iteration,ignore_index = True)
#    print(ProdH2ONdf)
#for H2 processes
    nadph2h2p = np.array(nh2den*pfh2h2p)
    noprods2=np.array(noprods2+nadph2h2p)
    madph2h2p = np.array(mh2den*pfh2h2p)
    morpods2 = np.array(moprods2+madph2h2p)
    nadph2hp = np.array(nh2den*pfh2hp)
    noprods1 = np.array(noprods1+nadph2hp)
    madph2hp = np.array(mh2den*pfh2hp)
    moprods1 = np.array(moprods1+madph2hp)
    nadph2vib = np.array(nh2den*pfh2vib)
    madph2vib = np.array(mh2den*pfh2vib)
    
#estimate H3+ production 
#assuming Dustin's ionospheric profile 
    photh2p = np.zeros_like(altgrd)
    photo2p = np.zeros_like(altgrd)
    photfh2hp = np.zeros_like(altgrd)
    photfo2op = np.zeros_like(altgrd)
    photh2op = np.zeros_like(altgrd)
    photfh2ohp = np.zeros_like(altgrd)
    photfh2oop = np.zeros_like(altgrd)
    if nutc > 165500:
        photh2p = photh2h2p
        photo2p = photo2o2p
        photfh2hp = photh2hp 
        photfo2op = photo2op 
        photfh2ohp = photh2ohp 
        photfh2oop = photh2oop 
        photh2op = photh2oh2op
    mphoth2p = np.zeros_like(altgrd)
    nadph2pden = np.array(H2iondensity(altgrd,nh2den,nadph2oh2p,nadph2h2p,photh2p))
    figionn = px.scatter(x=nadph2pden/1.E6, y=altgrd/1000., log_x=1, title='Ionospheric densities (cm^-3)',labels={"y":  "Altitude(km)", "x": "Density(cm^-3)"})
    figionn.add_scatter(x=nadph2pden/1.E6, y=altgrd/1000., mode='markers',name='Day H2+')
    figionn.add_scatter(x=electrondensity(altgrd)/1.E6, y=altgrd/1000., mode='markers', name='Day electrons')
    figionn.update_xaxes(exponentformat="E")
    figionn.show()
    nadph3p = np.array(nadph2pden*nh2den*2.e-15)
    madph2pden = np.array(H2iondensity(altgrd,mh2den,madph2oh2p,madph2h2p,mphoth2p))
    figionm = px.scatter(x=madph2pden/1.E6, y=altgrd/1000., log_x=1, title='Ionospheric densities (cm^-3)',labels={"y":  "Altitude(km)", "x": "Density(cm^-3)"})
    figionm.add_scatter(x=madph2pden/1.E6, y=altgrd/1000., mode='markers',name='Night H2+')
    figionm.add_scatter(x=electrondensity(altgrd)/1.E6, y=altgrd/1000., mode='markers', name='Night electrons')
    figionm.update_xaxes(exponentformat="E")
    figionm.show()
    madph3p = np.array(madph2pden*mh2den*2.e-15)
 
    
#Generate DataStructure for processes as a function of altitude
    ProdH2Ndf_iteration = pd.DataFrame(data={'UTC':nutc,'ALT':altgrd,'NADPH2H2P':nadph2h2p,'NADPH2HP':nadph2hp,'NADPH3P':nadph3p,'NADPH2vib':nadph2vib})
    ProdH2Mdf_iteration = pd.DataFrame(data={'UTC':nutc,'ALT':altgrd,'MADPH2H2P':madph2h2p,'MADPH2HP':madph2hp,'MADPH3P':madph3p,'MADPH2vib':madph2vib})
    ProdH2Ndf = ProdH2Ndf.append(ProdH2Ndf_iteration,ignore_index = True)
    ProdH2Mdf = ProdH2Mdf.append(ProdH2Mdf_iteration,ignore_index = True)
#   print(ProdH2Ndf)

               
#integrate all processes over the full altitude range and put integrated emissions in Rayleighs
                
#ionization
    ncpo2o2p = integrate.simpson((nadpo2o2p+photo2p),altgrd,even='avg')
    mcpo2o2p = integrate.simpson(madpo2o2p,altgrd,even='avg')
    ncpo2op = integrate.simpson((nadpo2op+photfo2op),altgrd,even='avg')
    mcpo2op = integrate.simpson(madpo2op,altgrd,even='avg')
    ncpo2opp = integrate.simpson(nadpo2opp,altgrd,even='avg')
    mcpo2opp = integrate.simpson(madpo2opp,altgrd,even='avg')
    ncph2oh2op = integrate.simpson((nadph2oh2op+photh2op),altgrd,even='avg')
    mcph2oh2op = integrate.simpson(madph2oh2op,altgrd,even='avg')
    ncph2oopp = integrate.simpson(nadph2oopp,altgrd,even='avg')
    mcph2oopp = integrate.simpson(madph2oopp,altgrd,even='avg')
    ncph2oh2p = integrate.simpson(nadph2oh2p,altgrd,even='avg')
    mcph2oh2p = integrate.simpson(madph2oh2p,altgrd,even='avg')
    ncph2ohp = integrate.simpson((nadph2ohp+photfh2ohp),altgrd,even='avg')
    mcph2ohp = integrate.simpson(madph2ohp,altgrd,even='avg')
    ncph2h2p = integrate.simpson((nadph2h2p+photh2p),altgrd,even='avg')
    mcph2h2p = integrate.simpson(madph2h2p,altgrd,even='avg')
    ncph2hp = integrate.simpson((nadph2hp+photfh2hp),altgrd,even='avg')
    mcph2hp = integrate.simpson(madph2hp,altgrd,even='avg')
    ncph3p = integrate.simpson(nadph3p,altgrd,even='avg')
    mcph3p = integrate.simpson(madph3p,altgrd,even='avg')
    ncphoth2p = integrate.simpson(photh2p,altgrd,even='avg')
    
#Dissociation of O2 only for now
    ncpo2dis = integrate.simpson(nadpo2dis,altgrd,even='avg')
    mcpo2dis = integrate.simpson(madpo2dis,altgrd,even='avg')
#for emission of 1304 and 1356
    nRo21304 = (integrate.simpson(nadpo21304,altgrd,even='avg'))/1.e+10
    mRo21304 = (integrate.simpson(madpo21304,altgrd,even='avg'))/1.e+10
    nRo21356 = (integrate.simpson(nadpo21356,altgrd,even='avg'))/1.e+10
    mRo21356 = (integrate.simpson(madpo21356,altgrd,even='avg'))/1.e+10
    nRh2o1304 = (integrate.simpson(nadph2o1304,altgrd,even='avg'))/1.e+10
    mRh2o1304 = (integrate.simpson(madph2o1304,altgrd,even='avg'))/1.e+10
    nRh2o1356 = (integrate.simpson(nadph2o1356,altgrd,even='avg'))/1.e+10
    mRh2o1356 = (integrate.simpson(madph2o1356,altgrd,even='avg'))/1.e+10
    
#Generate a data structure for the altitude inegrated production for each UTC
    CProdH2ONdf_iteration = pd.DataFrame(data={'UTC':nutc,'NCPH2OH2OP':ncph2oh2op,'NCPH2OOPP':ncph2oopp,'NCPH2OH2P':ncph2oh2p,'NCPH2OHP':ncph2ohp,'NRH2O1304':nRh2o1304,'NRH2O1356':nRh2o1356},index=[0])
    CProdH2OMdf_iteration = pd.DataFrame(data={'UTC':nutc,'MCPH2OH2OP':mcph2oh2op,'MCPH2OOPP':mcph2oopp,'MCPH2OH2P':mcph2oh2p,'MCPH2OHP':mcph2ohp,'MRH2O1304':mRh2o1304,'MRH2O1356':mRh2o1356},index=[0])
    CProdH2ONdf = CProdH2ONdf.append(CProdH2ONdf_iteration,ignore_index = True)
    CProdH2OMdf = CProdH2OMdf.append(CProdH2OMdf_iteration,ignore_index = True)
    CProdO2Ndf_iteration = pd.DataFrame(data={'UTC':nutc,'NCPO2O2P':ncpo2o2p,'NCPO2OP':ncpo2op,'NCPO2OPP':ncpo2opp,'NRO21304':nRo21304,'NRO21356':nRo21356,'NCPO2dis':ncpo2dis},index=[0])
    CProdO2Mdf_iteration = pd.DataFrame(data={'UTC':nutc,'MCPO2O2P':mcpo2o2p,'MCPO2OP':mcpo2op,'MCPO2OPP':mcpo2opp,'MRO21304':mRo21304,'MRO21356':mRo21356,'MCPO2dis':mcpo2dis},index=[0])
    CProdO2Ndf = CProdO2Ndf.append(CProdO2Ndf_iteration,ignore_index = True)
    CProdO2Mdf = CProdO2Mdf.append(CProdO2Mdf_iteration,ignore_index = True)
    CProdH2Ndf_iteration = pd.DataFrame(data={'UTC':nutc,'NCPH2H2P':ncph2h2p,'NCPH2HP':ncph2hp,'NCPH3P':ncph3p},index=[0])
    CProdH2Mdf_iteration = pd.DataFrame(data={'UTC':nutc,'MCPH2H2P':mcph2h2p,'MCPH2HP':mcph2hp,'MCPH3P':mcph3p},index=[0])
    CProdH2Ndf = CProdH2Ndf.append(CProdH2Ndf_iteration,ignore_index = True)
    CProddfH2M = CProdH2Mdf.append(CProdH2Mdf_iteration,ignore_index = True)

 
    
#Form useful column production rate sums for noon
#O2+ by O2 and H2O
    ncpo2p = ncpo2o2p    
    print("NOON","O2+ =", "{:.2e}".format(ncpo2p))
#H2+ by H2O and H2
    ncph2p = ncph2oh2p + ncph2h2p 
    nrh2h2o = ncph2h2p/ncph2oh2p
    print("NOON", "H2+ =", "{:.2e}".format(ncph2p), "PH2/PH2O =", "{:.2e}".format(nrh2h2o))
#mass 16 - water group - from H2O and O2
    ncpwg = ncph2oh2op + ncpo2op
    nrwgo2 = ncph2oh2op/ncpo2op
    print("NOON","WG+=", "{:.2e}".format(ncpwg), "WG/O2 =", "{:.2e}".format(nrwgo2))
#H3+ by H2+ + H2
    nrh3ph2p = ncph3p/(ncph2p-ncph3p)
    print("NOON","H3+=", "{:.2e}".format(ncph3p), "H3/H2 =", "{:.2e}".format(nrh3ph2p))
#H+ column
    ncphp = ncph2hp + ncph2ohp
    print("NOON H+=", "{:.2e}".format(ncphp)) 
#emission totals
    NR1304 = nRo21304 + nRh2o1304
    nro2h2o1304 = nRo21304/nRh2o1304
    NR1356 = nRo21356 + nRh2o1356
    nro2h2o1356 = nRo21356/nRh2o1356
    NR13561304 = NR1356/NR1304
    print(nutc)
    print('NOON','1304 in Rayleighs = ',"{:.2e}".format(NR1304), 'PO2/PH2O = ', "{:.2e}".format(nro2h2o1304))
    print('NOON','1356 in Rayleighs = ',"{:.2e}".format(NR1356), 'PO2/PH2O = ', "{:.2e}".format(nro2h2o1356))
    print('NOON','1356/1304 ratio = ', "{:.2e}".format(NR13561304))
#Form useful column production rate sums for midnight
#O2+ by O2 and H2O
    mcpo2p = mcpo2o2p 
    print("MIDNIGHT","O2+ =", "{:.2e}".format(ncpo2p))
#H2+ by H2O and H2
    mcph2p = mcph2oh2p + mcph2h2p
    mrh2h2o = mcph2h2p/mcph2oh2p
    print("MIDNIGHT", "H2+ =", "{:.2e}".format(ncph2p), "PH2/PH2O =", "{:.2e}".format(nrh2h2o))
#mass 16 - water group - from H2O and O2
    mcpwg = mcph2oh2op + mcpo2op
    mrwgo2 = mcph2oh2op/mcpo2op
    print("MIDNIGHT","WG+=", "{:.2e}".format(ncpwg), "WG/O2 =", "{:.2e}".format(nrwgo2))
#H3+ by H2+ + H2
    mrh3ph2p = mcph3p/(mcph2p-mcph3p)
    print("MIDNIGHT","H3+=", "{:.2e}".format(mcph3p), "H3/H2 =", "{:.2e}".format(mrh3ph2p))
#  H+
    mcphp = mcph2hp + mcph2ohp
    print("MIDNIGHT H+=", "{:.2e}".format(mcphp)) 
#emission totals
    MR1304 = mRo21304 + mRh2o1304
    mro2h2o1304 = mRo21304/mRh2o1304
    MR1356 = mRo21356 + mRh2o1356
    mro2h2o1356 = mRo21356/mRh2o1356
    MR13561304 = MR1356/MR1304
    print('MIDNIGHT','1304 in Rayleighs = ',"{:.2e}".format(MR1304), 'PO2/PH2O = ', "{:.2e}".format(mro2h2o1304))
    print('MIDNIGHT','1356 in Rayleighs = ',"{:.2e}".format(MR1356), 'PO2/PH2O = ', "{:.2e}".format(mro2h2o1356))
    print('MIDNIGHT','1356/1304 ratio = ', "{:.2e}".format(MR13561304))
#    print('ionization at midnight',mcpo2o2p,mcpo2op,mcpo2opp,mcph2oh2op,mcph2oo2p,mcph2oh2p,mcph2ohp,mcph2h2p,mcph2hp)
#    print('ionization at noon',ncpo2o2p,ncpo2op,ncpo2opp,ncph2oh2op,ncph2oo2p,ncph2oh2p,ncph2ohp,ncph2h2p,ncph2hp)
#    print('emission at midnight',mRo21304,mRo21356,mRh2o1304,mRh2o1356)
#    print('emission at noon',nRo21304,nRo21356,nRh2o1304,nRh2o1356)
#compare sum of incoming electrons to total ionization production
    elecsum = 0.
    ypassm2 = np.array(ypass*1e4)
    elecsum = (integrate.simpson(ypassm2,xpass,even='avg'))
    ynew = np.array(xpass*ypassm2)
    energysum = (integrate.simpson(ynew,xpass,even='avg'))
#    print(xpass,ypassm2,ynew)
#sum of ions produced
    nsumofions = ncpo2p + ncph2p + ncpwg + ncph2ohp + ncph2hp + ncpo2opp
    msumofions = mcpo2p + mcph2p + mcpwg + mcph2ohp + mcph2hp + mcpo2opp
    nenergyions = (25.+12.3)*nsumofions
    menergyions = (25.+12.3)*msumofions
# produced ions to incoming electrons
    nripein = nsumofions/elecsum
    mripein = msumofions/elecsum
    nenergyio = energysum/nenergyions
    menergyio = energysum/menergyions
    Tenergyflx = energysum*1.6e-16
    print('nsumofions',"{:.2e}".format(nsumofions), 'msumofions',"{:.2e}".format(msumofions),'esum',"{:.2e}".format(elecsum))
    print( 'Noon: Prod Ions / Electron In',"{:.2e}".format(nripein), 'Midnight: Prod Ions / Electron In',"{:.2e}".format(mripein))
    print( 'Noon: Energy in / Energy out',"{:.2e}".format(nenergyio), 'Midnight: Energy in / Energy Out',"{:.2e}".format(menergyio))
    print('Total Energy input in mW per meter squared',"{:.2e}".format(Tenergyflx))
    

In [23]:
# IONOSPHERE CODE
# These are subroutines for the ionospheric model portion of the code. It assumes long mean
# free paths for ions and electrons and a large Debye length of greater than 5km.
# The ion and electron transport is due to two forces - the polarization electric
# field and gravity. The inspiration for the derivation comes from S&N equations 
# 5.61 and 5.62, which involves the electron pressure gradient. Since the electrons 
# do not undergo sufficient collsions with neutrals to thermalize (check this) and the
# main source of electrons is secondary electrons produced by electron impact
# ionization then the effective 'temperature" to use to determine the pressure
# gradient is assumed to be the average secondary electron energy, which we assume 
# for now is similar between O2 (which we have), H2O, and H2. From these assumptions
# we can determine the (upward) velocity of an ion of mass mj acquired between grid
# point z(i+1) and z(i) as the square root {2[[(z(i+1)-z(i))e*voltage/mj]-r*g]}where 
# e*d(voltage)/dr = k*Te*d(ln(ne))/dr. This velocity can be used to determine the
# time spent in the altiude bin and determine how far the chemcial kinetis proceed
# before transport moves them on to the next bin, i.e, tbin=binsize/velocity of the
# jth ion. We assume that the slowest ion dominates the reaction kinetics as new
# faster ions are always streaming through when we assume steady state production.
# This time in the bin is compared to the loss frequency of the reaction to determine
# how far the reaction proceeds before transport takes over. Neutrals are considered
# stationary at this point in the development.
# The ions considered are S1=H+, S2=H2+, S3=H3+, S4=O+, S5=OH+, S6=H2O+, S7=H3O+,
# and S8=O2+. The production terms are of the form PSi and the loss terms of the form
# LSi. Rate constants from Luke Moore and Astrochemistry webiste and designated by
# number l,i.e Kl. The neutrals are N1=H2, N2=O2, N3=H2O, N4=H and N5=O are designated
# Nj. The first three are specified by Audrey's model and the H is calculated by local
#production balanced by outflow loss ? and may require iteration ?
# The reaction list is given below:
#    R1: N1 + EE --> S1 + N4 + esec + EE<
#    R2: N1 + hnu --> S1 + N4 + esec
#    R3: S1 + eth --> H + hnu
#    R4: S1 + N3 --> S6 + N4
#    R5: N1 + EE --> S2 + esec + EE<
#    R6: N1 + hnu --> S2 + eph
#    R7: S2 + eth --> 2N4
#    R7a: S2 + N4 --> N1 + S1
#    R8: S2 + N3 --> S7 + N4
#    R9: S2 + N3 --> S6 + N1
#    R10: S2 + N1 --> S3 + N4
#    R11: S3 + eth --> N1 + N4
#    R12: S3 + eth --> 3N4
#    R13: S3 + N3 --> S7 + N1
#    R14: N3 + hnu --> S6 + eph
#    R15: N3 + EE --> S6 + esec +EE<
#    R16: S6 + eth --> OH + N4
#    R17: S6 + eth --> N5 + N1
#    R18: S6 + N1 --> S7 + N4
#    R19: S6 + N3 --> S7 + OH
#    R20: N3 + hnu --> S1 + OH + eph
#    R21: N3 + EE --> S1 + OH + esec + EE<
#    R22: N3 + hnu --> S5 + N4 + eph
#    R23: N3 + EE --> S2 + O + esec + EE<
#    R24: S5 + N1 --> S6 + N4 
#    R25: S5 + N3 --> S7 + N5
#    R26: S7 + eth --> N3 + N4
#    R27: N3 + hnu --> S4 + N1 + eph
#    R28: 
#    R29: N2 + hnu --> S4 + N5 + eph
#    R30: N2 + EE --> S4 + N5 +esec + EE<
#    R31: S4 + H --> N5 + S1
#    R32: S4 + N3 --> S6 + N5
#    R33: S4 + N2 --> S8 + N5
#    R34: S4 + N1 --> S5 + N4
#    R35: N2 + hnu --> S8 + eph
#    R36: N2 + EE --> S8 + esec +EE<
#    R37: S8 + eth --> O + O

def reactionrates(T,Te,RDsun):
# This subroutine determines the reaction rates for reactions R1 to R36. Photon reactrions
# assume optically thin and are taekn from the Huebner compilation and are consistent with
# the earlier values used in the code. EE reaction rates depend on the electron energy spectrun
# used and aare calculated in the earlier section of the code - here we use quiet sun case. 
# These are KR1, KR5, KR15, KR21, KR23, KR30, and KR36.
    KR[0] = 0.
    tterm = Te/300
    KR[2] = 9.52e-9
    KR[3] = 1.91e-10*tterm**-0.70
    KR[4] = 6.9e-9
    KR[6] = 5.41e-8*(1/RDsun)**2
    KR[7] = 2.25e-6*(tterm**-0.40)
    KR[8] = 3.43e-9
    KR[9] = 3.87e-9
    KR[10] = 2e-9
    KR[11] = 7.62e-7*tterm**-0.50
    KR[12] = 9.7e-7*tterm**-0.50
    KR[13] = 5.3e-9
    KR[14] = 3.31e-7*(1/RDsun)**2
    KR[16] = 2.77e-6*tterm**-0.50
    KR[17] = 3.4e-6*tterm**-0.50
    KR[18] = 7.6e-10
    KR[19] = 1.85e-9
    KR[20] = 1.31e-8*(1/RDsun)**2
    KR[22] = 5.54e-8*(1/RDsun)**2
    KR[24] = 9.7e-10
    KR[25] = 2.9e-9
    KR[26] = 6.06e-6
    KR[27] = 5.85e-9*(1/RDsun)**2
    KR[30] = 1.1e-7*(1/RDsun)**2
    KR[31] = 5.66e-10*tterm**0.36*exp(8.60/T)
    KR[32] = 3.2e-9*tterm**-0.50
    KR[33] = 1.9e-11
    KR[34] = 1.7e-9
    KR[35] = 4.67e-7*(1/RDsun)**2
    KR[37] = 1.9e-7*tterm**-0.50
    return KR

def chemprod(nden,iden):
# Calculates all production terms except electron impact energy dependent prod rates
    prod[0] = 0.
    prod[1] = nden[1]*(KR[1]+KR[2])+nden[3]*KR[20]
    prod[2] = nden[1]*(KR[5]+KR[6])+nden[3]*KR[23]
    prod[3] = iden[2]*nden[1]*KR[10]
    prod[4] = nden[2]*(KR(29)+KR[30])+nden[3]*KR[27]
    prod[5] = nden[3]*KR[22]+iden[4]*nden[1]*KR[34]
    prod[6] = nden[3]*(KR[14]+KR[15])+iden[1]*nden[3]*KR[4]+iden[2]*nden[3]*KR[9]
    prod[7] = nden[1]*iden[6]*KR[18]+nden[3]*iden[3]*KR[13]+nden[3]*iden[6]*KR[19]
    prod[8] = nden[2]*(KR[33]*iden[4]+KR[35]+KR[36])
    return prod

def lossfreq(nden,elec):
# Calculates loss frequency for ion species
    lfreq[0] = 0.
    lfreq[1] = elec*KR[3]+nden[3]*KR[4]
    lfreq[2] = elec*KR[7]+nden[3]*(KR[8]+KR[9])
    lfreq[3] = elec*(KR[11]+KR[12])+nden[3]*KR[13]
    lfreq[4] = nden[1]*KR[34]+nden[2]*KR[33]+nden[3]*KR[32]+nden[4]*KR[31]
    lfreq[5] = nden[1]*KR[24]+nden[3]*KR[25]
    lfreq[6] = elec*(KR[16]+KR[17])+nden[1]*KR[18]*nden[3]*KR[19]
    lfreq[7] = elec*KR[26]
    lfreq[8] = elec*KR[37]
    return lfreq

def gravaccel(alt):
# Solves Ganymede gravitational accleration at altitude alt in cgs units
    gg = 142.8
    gradius = 2.63341e+8
    galt = gg*(gradius/(alt+gradius))**2
    return galt

def tstep(y,d,v0,alt):
# Solves quadratic for time step to traverse a radial grid point
    b = v0
# for now Efieldi preset to zero for the call
    Efieldi = 0.
    accel = defaccel(y,alt,d,Efieldi)
    a = accel/2.
    term = sqrt(b*b-4.*a*d)
    tp = (-b+term)/(2.*a)
    tm = (-b-term)/(2.*a)
    if tp>0.: 
        t = tp   
    else:
        t = tm
    v = v0+accel*t
    tv = np.array([t,v])
    return tv

def defaccel(y,alt,d,Efieldi):
# Defines the density weighted acceleration over a grid spacing using an electric
# field input term and gravity in cgs units
# Note ms0 array must contain unit mass of ion species vector y
    amu = 1.66e-24
    q0 = 2.88e9
    gterm = -gravaccel(alt)
    iter = len(y)-1
    stot = 0.
    for i in range(iter):
        stot = stot + y(i)
    ms0 = np.array([1.,2.,3.,16.,17.,18.,19.])
    mwst = 0.
    for i in range(iter):
        mwst = mwst + (y[i)]/stot)*ms0[i]
    mavg  = mswt*amu
#    eterm = q0*Efieldi/mavg
# For now set eterm to be -2*gterm
    eterm = -2.*gterm
    accel = gterm +  eterm
    return accel                  

def fode(y,t,itgrd,ivgrd,idgrd,nden,elec):
# Provides the function definition for the coupled set of equations for the
# evolution of the ionospheric densities during ion outflow
    naltgrdpts=len(nden-2)
    for i in range(naltgrdpts):
        itest[i]=itgrd[i,i]
        itest[i+1]=itgrd[i+1,i+1]
        if (t.ge.itest[i].and.lt.itest[i+1]):
            nbalt=i
        else if (t.eq.itest[i+1]):
            nbalt=i+1
    dtmp=idgrd[nbalt,nbalt]
    vl=ivgrd[nbalt,nbalt]
    vu=ivgrd[nbalt+1,ivgrd[nbalt+1]]
    fa = (vl+vu)/dtmp
    fb = (vu-vl)/dtmp
    ndentmpl=np.array([nden[nbalt,1],[nbalt,2],[nbalt,3]])
    ndentmpu=np.array([nden[nbalt+1,1],[nbalt+1,2],[nbalt+1,3]])
    electmpl=elec[nbalt]
    electtmpu=elec[nbalt+1]
# Calculate coupled chemistry
    prodl = np.array(chemprod(ndentmpl,y))
    lfreql = np.array(lossfreq(ndentmpl,electmpl))
    produ = np.array(chemprod(ndentmpu,y))
    lfrequ = np.array(lossfreq(ndentmpu,electmpu))
    prodavg=(prodl+produ)/2.
    lfreqavg=(lfrql+lfrqu)/2.
    yold = np.array([soln[itgrd[nbalt,nbalt],0],soln[itgrd[nbalt,nbalt],1]],soln[itgrd[nbalt,nbalt],2],soln[itgrd[nbalt,nbalt],3],soln[itgrd[nbalt,nbalt],4],soln[itgrd[nbalt,nbalt],5],soln[itgrd[nbalt,nbalt],6],soln[itgrd[nbalt,nbalt],7])
    iter = len(y) - 1
    for i in range(iter):
        prodterm = np.array(prodavg-((fa+fb)*yold)
        lfterm = -(lfrqavg*y-(fa+fb))
        function = np.array(prodterm+lfterm)
    return function

def initialize(KR,nbalt,nden,elec):
# Initialize with a simple phtochemical equilibrium value near the surface
    Si[0] = 0.
    Si[1] = (KR[2]*nden[1,nbalt] + KR[20]*nden[2,nbalt])/(KR[3]*elec[0])
    Si[2] = KR[6]*nden[1,nbalt]/(KR[7]*elec[0])
    Si[3] = 0.0
    Si[4] = (KR[27]*nden[3,nbalt]+KR[30]*nden[2,nbalt])/(KR[34]*nden[1,nbalt])
    Si[5] = RK[22]*nden[3,nbalt]/(KR[24]*nden[1,nbalt])
    Si[6] = KR[14]*dden[3,nbalt]/(KR[16]*elec[0])
    Si[7] = 0.0
    Si[8] = RK[35]*nden[2,nbalt]/(KR[37]*elec[0])
    return Si

# Define the times and velocities associated with the altitude grid points
naltgrdpts = len(altgrd)
d = np.zeros_like(altgrd)
for i in range(naltgrdpts):
    if i=0:
        r0=0.
        v0=0.
    else:
        r0=altgrd[i-1]
    d=(altgrd[i]-r0)
    alt=altgrd[i]
    tv=                       
    itgrd=np.array([i,tv[1]])
    ivgrd=np.array([i,tv[2]])
    idgrd=np.array([i,d])

from scipy.integrate import odeint
import numpy as np

# solve the system dy/dt = f(y, t)


# initial conditions
yo = np.array(initialize(reactionrates(100.,100.,27.04),nh2den,no2den,nh2oden,elec)         

# solve the DEs
soln = odeint()
S[1] = soln[:, 0]
S[2] = soln[:, 1]
S[3] = soln[:, 2]
S[4] = soln[:, 3]
S[5] = soln[:, 4]
S[6] = soln[:, 5]
S[7] = soln[:, 6]
S[8] = soln[:, 7]
print(S)
                  
                  
                  
              
              
              
    
    

        
    
    
        
    
    
    
    


    
                     

SyntaxError: invalid syntax (<ipython-input-23-acb6c08c3591>, line 212)

In [1]:
             

#'Calculate Dissociative Excitation of O2 with Shemansky model Kanik et al, JGR, pp. 12-7to 12-8'
#O2 DisEx comes from Kanik et al.JGR, 108(E11),5126, doi:10.1029/2000JE001423,2003 and is a fit provided by function DS
#Def DSde(energy):
"""Calculate Dissociative Excitation of O2 with Shemansky model Kanik et al, JGR, pp. 12-7to 12-8"""
#    X1304l = energy/14.67
#   X1304u = energy/40.20
#    X1356l = energy/14.26
#    X1356u = energy/37.50
#    Rh1304 = 0.698813084
#    Rh1356 = 0.672014942
#    c04l0 = 0.
#    c04l1 = -0.01805967
#    c04l2 = 0.
#    c04l3 = 0.
#    c04l4 = -0.275652
#    c04l5 = 0.1830147
#    c04l6 = -0.1830147
#    c04l7 = 0.07549554
#    c04l8 = 0.15136
#    SUM04l = c04l1*(X1304l-1.)*exp(-c04l8*X1304l)+c04l2*(X1304l-1.)*exp(-2.*c04l8*X1304l)+c04l3*(X1304l-1.)*exp(-3.*c04l8*X1304l)+c04l4*(X1304l-1.)*exp(-4.*c04l8*X1304l)
#    omX1304l = c04l0(1.-1/X1304l)/X1304l**2 + SUM04l + c04l5 + c04l6/X1304l + c04l7*ln(X1304l)
#    Q1304l = omX1304l/(Rh1304*X1304l)
#    c04u0 = 0.
#    c04u1 = -0.00135933
#    c04u2 = 0.
#    c04u3 = 0.
#    c04u4 = -0.020748
#    c04u5 = 0.0137753
#    c04u6 = -0.0137753
#    c04u7 = 0.00568246
#    c04u8 = 0.15136
#    SUM04u = c04u1*(X1304u-1.)*exp(-c04u8*X1304u)+c04u2*(X1304u-1.)*exp(-2.*c04u8*X1304u)+c04u3*(X1304u-1.)*exp(-3.*c04u8*X1304u)+c04u4*(X1304u-1.)*exp(-4.*c04u8*X1304u)
#    omX1304l = c04u0(1.-1/X1304u)/X1304u**2 + SUM04u + c04u5 + c04u6/X1304u + c04u7*ln(X1304u)
#    Q1304u = omX1304u/(Rh1304*X1304u)
#    Q1304 = (Q1304l*0.93+Q1304u*0.07)*8.8e-17
#    c56l0 = 0.04800065
#   c56l1 = -0.0778563
#    c56l2 = -0.0392844
#    c56l3 = 0.
#    c56l4 = -0.532608
#    c56l5 = 0.474354
#    c56l6 = -0.474354
#    c56l7 = 0.1628775
#    c56l8 = 0.16596
#    SUM56l = c56l1*(X1356l-1.)*exp(-c56l8*X1304l)+c56l2*(X1356l-1.)*exp(-2.*c56l8*X1356l)+c56l3*(X1356l-1.)*exp(-3.*c56l8*X1356l)+c56l4*(X1356l-1.)*exp(-4.*c56l8*X1356l)
#    omX1356l = c56l0(1.-1/X1356l)/X1356l**2 + SUM56l + c56l5 + c56l6/X1356l + c56l7*ln(X1356l)
#    Q1356l = omX1356l/(Rh1356*X1356l)
#    c56u0 = 0.00252635
#    c56u1 = -0.0040977
#    c56u2 = -0.0020676
#    c56u3 = 0.
#    c56u4 = 0.028032
#    c56u5 = -0.024966
#    c56u6 = -0.024966
#    c56u7 = 0.0085725
#    c56u8 = 0.16596
#    SUM56u = c56u1*(X1356u-1.)*exp(-c56u8*X1356u)+c56u2*(X1356u-1.)*exp(-2.*c56u8*X1356u)+c56u3*(X1356u-1.)*exp(-3.*c56u8*X1356u)+c56u4*(X1356u-1.)*exp(-4.*c56u8*X1356u)
#    omX1356l = c56u0(1.-1/X1356u)/X1356u**2 + SUM56u + c56u5 + c56u6/X1356u + c56u7*ln(X1356u)
#    Q1356u = omX1356u/(Rh1356*X1356u)
#    Q1356 = (Q1356l*0.95+Q1356u*0.05)*8.8e-17
#Return Q1304,Q1356
    
    


              

'Calculate Dissociative Excitation of O2 with Shemansky model Kanik et al, JGR, pp. 12-7to 12-8'

In [ ]:
nutc=165200
#np.array(dfJADE_sub["E","UTC":nutc],dtype=float)
print(dfJADE_sub)